### Lib Imports 

In [1]:
import re
import socket
import json
import tweepy
import settings
import pandas
from tweepy import OAuthHandler, Stream
from tweepy.streaming import StreamListener
from pandas.io.json import json_normalize

### KEYS for TWITTER API Auth 

In [2]:
consumer_key = settings.ENV['CONSUMER_KEY']
consumer_secret = settings.ENV['CONSUMER_SECRET']
access_token = settings.ENV['ACCESS_KEY']
access_secret = settings.ENV['ACCESS_SECRET']

### Define regex pattern 

In [3]:
pattern = re.compile(r"#?AZUL4|Azul|#?BBAS3|#?BB|BANCO DO BRASIL|#?BBDC[34]|BRADESCO|#?BBSE3|BBSE|BB SEGURIDADE|#?BRAP4|BRADESPAR|#?BRDT3|#?PETR[34]?|BR DISTRIBUIDORA|PETROBR[AÁ]S|#?BRFS3|BRF|#?BRKM5|BRASKEM|#?BRML3|BR MALLS|#?BTOW3|B2W|#?CCRO3|CCR|#?CIEL3|CIELO|#?CMIG4|CEMIG|#?CSAN3|COSAN|#?CSNA3|CSN|#?CVCB3|CVC|#?CYRE3|CYRELA|#?ECOR3|ECORODOVIAS|#?B3SA3|B3|#?EGIE3|ENGIE BRASIL|ENGIE|#?ELET[36]|ELETROBRAS|#?EMBR3|EMBRAER|#?ENBR3|Energias|#?EQTL3|Equatorial|#?ESTC3|ESTACIO|#?FLRY3|FLEURY|#?GGBR4|GOAU4|GERDAU|#?GOLL4|Gol|#?HYPE3|HYPERA|#?IGTA3|IGUATEMI|#?IRBR3|IRB BRASIL|IRB BRASIL RE|IRB|#?ITSA4|ITAÚSA|#?ITUB4|ITAÚ UNIBANCO|ITA[ÚU]|#?JBSS3|JBS|#?KLBN11|KLABIN|#?KROT3|KROTON|#?LAME4|Americanas|Lojas Americanas|#?LREN3|LOJAS RENNER|RENNER|#?MGLU3|MAGAZINE LUIZA|#?MRFG3|MARFRIG|#?MRVE3|MRV|#?MULT3|MULTIPLAN|#?NATU3|Natura|#?PCAR[34]?|PÃO DE AÇÚCAR|#?QUAL3|QUALICORP|#?RADL3|RAIA DROGASIL|#?RAIL3|Rumo|#?RENT3|Localiza|#?SANB11|SANTANDER|#?SBSP3|SABESP|#?SMLS3|SMILES|#?SUZB3|SUZANO|#?TAEE11|TAESA|#?TIMP3|TIM|#?UGPA3|ULTRAPAR|#?USIM5|USIMINAS|#?VALE3?|Vale|#?VIVT4|Telefônica Brasil|Vivo|#?VVAR3|VIA VAREJO|#?WEGE3|WEG",  re.IGNORECASE)

###  TWITTER STREAM LISTENER 

In [4]:
class TweetListener(StreamListener):
    def __init__(self,csocket):
        self.client_socket = csocket 
    def on_data(self, data):
        try:
            tweet = json.loads(data)
            
            # remove links
            tweet['text'] = re.sub(r"http\S+", "", tweet['text'])
            
            # apply search regex
            if re.search(pattern, tweet['text']):
                print(tweet['text'].encode('utf-8'))
                
                # save raw_data
                with open('raw_data.json', 'a') as tf:
                    tf.write(data)
                
                #send data via socket
                self.client_socket.send(tweet['text'].encode('utf-8'))
            else:
                #print("match not found on: ") 
                #print(tweet['text'].encode('utf-8'))
                #send data via socket
                #self.client_socket.send(tweet['text'].encode('utf-8'))
                pass
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)
        return True
        

### Define accounts to fetch the data

In [5]:
#AgenciaEstado : 75289487 | leiamoneytimes: 51150679 | valor_ri: 104214965 | valorinveste: 1113533606247436289 | valoreconomico: 32529120 | EstadaoEconomia: 78774644 | InvestingBrasil: 49292227 | infomoney: 59773459 | ReutersBrasil: 17711232 | colunadobroad: 791391509631336448 
follow_busca = ["75289487", "51150679", "104214965", "1113533606247436289", "32529120", "78774644", "49292227", "59773459", "17711232", "791391509631336448"]

### API auth to send data

In [6]:
def sendData(c_socket):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    twitter_stream = Stream(auth, TweetListener(c_socket))
    twitter_stream.filter(languages=["pt"],follow=follow_busca)

### Define the socket 

In [7]:
s = socket.socket()
host = "127.0.0.1"
port = 8181
s.bind((host, port))

print("Listening on port: %s" % str(port))

Listening on port: 8181


### Listen the open socket

In [8]:
s.listen(1)
c, addr = s.accept()

print("Received request form " + str(addr))

Received request form ('127.0.0.1', 49618)


### Send the data and show 

In [9]:
sendData( c )

b'RT @leiamoneytimes: Fiocruz avalia condi\xc3\xa7\xc3\xb5es de trabalho na sa\xc3\xbade durante a pandemia   #Brasil #Conass #Coronav\xc3\xadrus\xe2\x80\xa6'
b'Banc\xc3\xb5es ir\xc3\xa3o pagar mais R$ 6 bilh\xc3\xb5es em impostos. Quem ir\xc3\xa1 "sofrer" mais? #bancos #empresas #ITUB4 #BBDC4 #SANB11\xe2\x80\xa6 '
b'@leiamoneytimes Quem ir\xc3\xa1 "sofrer" mais?\nacionista de BBDC4'
b'Trinta startups v\xc3\xa3o para a fase final de edital da Petrobras e Sebrae\n'
b'A\xc3\xa7\xc3\xb5es da Weg fecham em alta de 13,9% ap\xc3\xb3s divulga\xc3\xa7\xc3\xa3o de balan\xc3\xa7o \xc3\xa0 prova de coronav\xc3\xadrus '
b'RT @valorinveste: Trinta startups v\xc3\xa3o para a fase final de edital da Petrobras e Sebrae\n'
b'RT @valorinveste: Trinta startups v\xc3\xa3o para a fase final de edital da Petrobras e Sebrae\n'
b'RT @leiamoneytimes: PIB cai menos que o previsto, diz secret\xc3\xa1rio especial de Fazenda  #Coronav\xc3\xadrus #Economia #IOF #J\xe2\x80\xa6'
b'A\xc3\xa7\xc3\xb5es da B2W encerram 

b'RT @leiamoneytimes: Internacional EUA d\xc3\xa3o \xc3\xa0 China 72 horas para fechar consulado de Houston  #China #Coronav\xc3\xadrus #EU\xe2\x80\xa6'
b'RT @RicardoBomfim91: Empresa de alta tecnologia no mercado brasileiro? Conhe\xc3\xa7a a Weg nesta mat\xc3\xa9ria que eu escrevi para o @infomoney e saiba\xe2\x80\xa6'
b'Projeto dos tr\xc3\xaas maiores bancos do Pa\xc3\xads prev\xc3\xaa financiamento para viabilizar investimentos e desenvolver a infraestr\xe2\x80\xa6 '
b'No encontro de Bradesco, Ita\xc3\xba e Santander com membros do governo, foram discutidas medidas sobre preserva\xc3\xa7\xc3\xa3o e bioe\xe2\x80\xa6 '
b'RT @infomoney: Bradesco, Ita\xc3\xba e Santander lan\xc3\xa7am plano para promover desenvolvimento sustent\xc3\xa1vel na Amaz\xc3\xb4nia '
b'Marfrig divulgar\xc3\xa1 plano de a\xc3\xa7\xc3\xa3o socioambiental nesta quinta-feira #MRFG3 #empresas #frigor\xc3\xadficos #sustentabilidade\xe2\x80\xa6 '
b'RT @RicardoBomfim91: Empresa de alta tecnologia no mercado brasileiro?

b'RT @leiamoneytimes: Oi: Highline do Brasil bate Vivo, Tim e Claro e d\xc3\xa1 a melhor oferta pelos ativos m\xc3\xb3veis #oibr3 #oi '
b'@leiamoneytimes Est\xc3\xa1 na hora de reformular a qualidade dos carros, tamb\xc3\xa9m.'
b'RT @InvestingBrasil: Oi assina acordo de exclusividade com a Highline, da Digital Colony, para negociar sua rede m\xc3\xb3vel\n\n#OIBR3 #VIVT4 #TIMP\xe2\x80\xa6'
b'@leiamoneytimes @sergiosanita Amanh\xc3\xa3 abre com gap em 2 reais.'
b'RT @InvestingBrasil: Oi assina acordo de exclusividade com a Highline, da Digital Colony, para negociar sua rede m\xc3\xb3vel\n\n#OIBR3 #VIVT4 #TIMP\xe2\x80\xa6'
b'RT @InvestingBrasil: Oi assina acordo de exclusividade com a Highline, da Digital Colony, para negociar sua rede m\xc3\xb3vel\n\n#OIBR3 #VIVT4 #TIMP\xe2\x80\xa6'
b'RT @leiamoneytimes: Oi: Highline do Brasil bate Vivo, Tim e Claro e d\xc3\xa1 a melhor oferta pelos ativos m\xc3\xb3veis #oibr3 #oi '
b'RT @leiamoneytimes: JHSF vai reabrir Hotel Fasano Angra dos Reis em 15

KeyboardInterrupt: 

### Store raw_data in HDFS

In [10]:
! /home/vineasouza/hadoop/bin/hadoop fs -put -f /home/vineasouza/Documentos/GitLab/pvi2019080000223/codes/final/raw_data.json /user/input/sentiment_result